<a href="https://colab.research.google.com/github/AvisR/Drawdown/blob/main/Drawdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question - Find Nth largest drawdowns given the return series and N

Defining Drawdown - Largest negative movement in percentage. 2nd largest drawdown will be the 2nd largest negative movement in percentage. 

Drawdown = (Peak - Trough)/Peak

Return series = rets

N = N

---



In [14]:
#INPUTS - Create a list of random returns from (-5% and 10%)
#This will not be needed if we have a list of our own returns.
#II = Initial Investment

II = 1
N = 1

import random
rets = random.sample(range(-50,100),100)
rets = [x/1000 for x in rets]

print(rets)

[0.024, 0.047, 0.016, -0.008, 0.022, -0.014, 0.011, -0.001, 0.091, -0.04, 0.041, 0.083, 0.04, 0.012, 0.027, 0.005, 0.0, -0.002, 0.053, -0.019, 0.035, -0.028, 0.006, -0.006, -0.025, -0.029, 0.092, -0.012, 0.02, 0.077, 0.001, -0.026, 0.004, 0.008, -0.047, 0.074, 0.088, -0.01, 0.002, 0.043, 0.067, -0.015, 0.007, 0.068, -0.017, 0.033, 0.048, 0.028, -0.005, -0.032, -0.003, 0.084, -0.03, 0.031, 0.079, 0.003, -0.036, 0.062, -0.018, -0.035, 0.036, 0.086, -0.022, -0.043, -0.044, 0.042, 0.096, 0.025, 0.051, 0.05, 0.057, 0.098, 0.095, 0.087, 0.01, 0.094, -0.009, 0.049, -0.024, 0.059, 0.044, 0.06, 0.026, 0.052, 0.056, 0.045, -0.034, 0.078, 0.034, 0.029, 0.097, 0.07, 0.081, 0.065, 0.082, 0.014, 0.055, -0.031, -0.045, 0.099]


In [15]:
#Step 1 = Calculate value of portfolio.
#Initial investment = 1, value of the portfolio is calculated using initial investment and cumulative product.
#For eg. Value of the portfolio at the end of period 10 would be (1*(1+p1 return)*(1+p2 return)*....(1+p10 return))

cmfactor = [0]*(len(rets))

def portvalue(rets):
    mfactor = [1+x for x in rets]
    for i in range(0,len(rets)):
      if i == 0:
        cmfactor[i] = II * mfactor[i]
      else:
        cmfactor[i] = cmfactor[i-1]*mfactor[i]
      cmfactor[i] = round(cmfactor[i],4)
    return(cmfactor)

portvalue = portvalue(rets)

print(portvalue)


[1.024, 1.0721, 1.0893, 1.0806, 1.1044, 1.0889, 1.1009, 1.0998, 1.1999, 1.1519, 1.1991, 1.2986, 1.3505, 1.3667, 1.4036, 1.4106, 1.4106, 1.4078, 1.4824, 1.4542, 1.5051, 1.463, 1.4718, 1.463, 1.4264, 1.385, 1.5124, 1.4943, 1.5242, 1.6416, 1.6432, 1.6005, 1.6069, 1.6198, 1.5437, 1.6579, 1.8038, 1.7858, 1.7894, 1.8663, 1.9913, 1.9614, 1.9751, 2.1094, 2.0735, 2.1419, 2.2447, 2.3076, 2.2961, 2.2226, 2.2159, 2.402, 2.3299, 2.4021, 2.5919, 2.5997, 2.5061, 2.6615, 2.6136, 2.5221, 2.6129, 2.8376, 2.7752, 2.6559, 2.539, 2.6456, 2.8996, 2.9721, 3.1237, 3.2799, 3.4669, 3.8067, 4.1683, 4.5309, 4.5762, 5.0064, 4.9613, 5.2044, 5.0795, 5.3792, 5.6159, 5.9529, 6.1077, 6.4253, 6.7851, 7.0904, 6.8493, 7.3835, 7.6345, 7.8559, 8.6179, 9.2212, 9.9681, 10.616, 11.4865, 11.6473, 12.2879, 11.907, 11.3712, 12.4969]


In [16]:
# Step 2 = Find all difference. 
# The idea is to create a matrix which contains differences betweem all portfolio value. 
# For eg - The first row of the matrix will contain difference between initial investment and the subsequent values. 
# This can be optimized by running the for loop in reverse(i.e from 99 to 0) and stopping the iteration as soon we find a future drawdown greater than the current drawdown.  
# For eg. Value at t0 = 1. Value at time 10 = 1.3 Value at time 23 = .5. The drawdown from time 10 will always be greater than the drawdown from t0. So we need to evaluate only till t9 for t0.
# This is current done by running a for loop where I compare each element with elements below in the same column. If they are smaller(larger drawdown), the element will reset to 0 now and be removed in Step 3.  


#Matrix of differences

ddmatrix = [[0 for i in range(len(rets))] for a in range(len(rets))]

def diffmat(rets):
    for i in range(len(rets)):
      if i == 0:
        for a in range(len(rets)):
         ddmatrix[i][a]= (portvalue[a]-II)/II
      else:
        for a in range(len(rets)):
         ddmatrix[i][a]= (portvalue[a]-portvalue[i-1])/portvalue[i-1]
    return(ddmatrix)

ddmatrix = diffmat(rets)

#Making it upper triangle matrix. 
for i in range(len(ddmatrix)):
  for a in range(len(ddmatrix)):
    if a < i:
      ddmatrix[i][a] = 0
    else:
      ddmatrix[i][a] = round(ddmatrix[i][a],4)
    for b in range(b+1,len(ddmatrix)):
      if ddmatrix[i][a] < ddmatrix[i][b]:
        ddmatrix[i][a] = 0


In [18]:
#Step 3 = Find the largest drawdown for each date. 

mddmatrix = [[0 for i in range(3)] for a in range(len(rets))]

def min_value(ddmatrix):
      l = len(ddmatrix)
      minlist = [min((ddmatrix)[i]) for i in range(l)]
      mdd = minlist
      return (minlist)

mdd = min_value(ddmatrix)

print(list(mdd))

[0.024, 0, 0, -0.008, 0, -0.014, 0, -0.001, 0, -0.04, 0, 0, 0, 0, 0, -0.0133, -0.0181, -0.0181, -0.0162, -0.0657, -0.0476, -0.0798, -0.0533, -0.059, -0.0533, -0.029, 0, -0.012, 0, 0, -0.0596, -0.0606, -0.0355, -0.0393, -0.047, 0, 0, -0.01, 0, 0, 0, -0.015, 0, 0, -0.017, 0, 0, -0.0128, -0.0397, -0.0349, -0.003, 0, -0.03, 0, 0, -0.0331, -0.036, 0, -0.0524, -0.035, 0, -0.0283, -0.1052, -0.0851, -0.044, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.009, 0, -0.024, 0, 0, 0, 0, 0, 0, 0, -0.034, 0, 0, 0, 0, 0, 0, 0, 0, -0.01, -0.0237, -0.0746, -0.045, 0]


In [22]:
#Step 4 - Build MDD Matrix(Maximum drawdown for each date along with the time period). 
#Remove 0s and overlapping drawdowns and return Nth drawdown. 

mddmatrix = [[0 for i in range(3)] for a in range(len(rets))]

#Build MDD matrix and remove Zero drawdowns
for x in range(len(mddmatrix)):
  mddmatrix[x][0] = mdd[x]
  for a in range(len(rets)):
    if ddmatrix[x][a] == mddmatrix[x][0]:
       mddmatrix[x][1] = x
       mddmatrix[x][2] = a+1
    if mddmatrix[x][0] >= 0: 
       mddmatrix[x][1] = 0
       mddmatrix[x][2] = 0

mddmatrix = ([i for i in mddmatrix if any(i)])
mddmatrix.sort()

#Remove overlapping drawdowns
def unique(mddmat): 
    found = set()
    for item in mddmat:
        if item[2] not in found:
            yield item
            found.add(item[2])

mddmatrix = list(unique(mddmatrix))
#nth drawdown
def ndraw(N):
  if N > len(mddmatrix):
    return(print("We do not have", N, "drawdown"))
  elif mddmatrix[1] == 0:
    return(print("We do not have", N, "drawdown"))
  else:
    return(mddmatrix[N-1])

ndrawdown = ndraw(N)

print(N, "th largest drawdown is", ndrawdown[0], "and it occurs between", ndrawdown[1], "and", ndrawdown[2])
print(mddmatrix)

1 th largest drawdown is -0.1052 and it occurs between 62 and 65
[[-0.1052, 62, 65], [-0.0798, 21, 26], [-0.0746, 97, 99], [-0.0606, 31, 35], [-0.0524, 58, 60], [-0.04, 9, 10], [-0.0397, 48, 51], [-0.036, 56, 57], [-0.034, 86, 87], [-0.03, 52, 53], [-0.024, 78, 79], [-0.017, 44, 45], [-0.015, 41, 42], [-0.014, 5, 6], [-0.012, 27, 28], [-0.01, 37, 38], [-0.009, 76, 77], [-0.008, 3, 4], [-0.001, 7, 8], [0.024, 0, 0]]
